In [29]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

#remove limit of dataframe display
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)



In [30]:
# take csv file and return a dataframe
path_file = 'Android_Malware_Benign.csv'
dataset = pd.read_csv(path_file)
# randomize order of rows with a seed
dataset = dataset.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
# print(dataset.head(5))
# print(dataset.info())
# print(test)

dataset_prepprocess = dataset.copy()

for column in dataset.columns:
    if dataset[column].nunique()== 1:
        print(column)
        dataset_prepprocess.drop(column, axis=1, inplace=True)


ACCESS_PROVIDER
ACCESS_SERVICE
ACCESS_SHARED_DATA
activityCalled
ACTIVITY_RECOGNITION
ANT
ANT_ADMIN
AUTORUN_MANAGER_LICENSE_MANAGER
AUTORUN_MANAGER_LICENSE_SERVICE(.autorun)
BIND_GET_INSTALL_REFERRER_SERVICE
CHANGE_DISPLAY_MODE
CHECK_LICENSE
DATABASE_INTERFACE_SERVICE
DOWNLOAD_SERVICE
EXTENSION_PERMISSION
FULLSCREEN.FULL
GOOGLE_AUTH
GOOGLE_PHOTOS
JPUSH_MESSAGE
MAPS_RECEIVE
MESSAGE
PERMISSION
PERMISSION_RUN_TASKS
PLUGIN
READ
READ_ATTACHMENT
READ_AVESTTINGS
READ_CONTENT_PROVIDER
READ_DATA
READ_DATABASES
READ_GMAIL
READ_GSERVICES
READ_MESSAGES
RECEIVE
RECEIVE_SIGNED_DATA_RESULT
RESPOND
REQUEST
SEND
WRITE
WRITE_AVSETTING
WRITE_DATA
WRITE_DATABASES
android.permission.PROCESS_INCOMING_CALLS
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4464 entries, 0 to 4463
Columns: 285 entries, ACCESS_ALL_DOWNLOADS to Label
dtypes: int64(284), object(1)
memory usage: 9.7+ MB
None


In [ ]:
print(dataset_prepprocess.info())
print(dataset.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4464 entries, 0 to 4463
Columns: 285 entries, ACCESS_ALL_DOWNLOADS to Label
dtypes: int64(284), object(1)
memory usage: 9.7+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4464 entries, 0 to 4463
Columns: 328 entries, ACCESS_ALL_DOWNLOADS to Label
dtypes: int64(327), object(1)
memory usage: 11.2+ MB
None


In [ ]:
print(dataset_prepprocess.head())